In [1]:
import os

In [2]:
os.chdir("..")

In [3]:
%pwd

'e:\\wine_mlflow'

In [32]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE:str
    unzip_dir:Path
    all_schema:dict

In [33]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_valiation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_dir=config.unzip_dir,
            all_schema=schema,
        )

        return data_validation_config
    
        

In [34]:
import os
import pandas as pd
from src.mlproject import logging

class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_data(self)->bool:
        try:
            validation_status=None
            logging.info("enterd validation")

            df=pd.read_csv(self.config.unzip_dir)
            all_cols=list(df.columns)
            
            all_schema=self.config.all_schema.keys()

            for cols in all_cols:
                if cols not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e


In [36]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_valiation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_data()
except Exception as e:
    raise e
